# Import Constraint library and JSON

In [6]:
from constraint import Problem, AllDifferentConstraint
import json

# Read txt file

In [7]:
# ==========================================
# Ler ficheiro txt
# ==========================================

def lerDados(filename):
    dados = {"cc" : {}, "olw" : [], "dsd": {}, "tr" : {}, "rr" : {}, "oc" : {}}
    secao = None

    with open(filename, "r", encoding="utf-8") as file:
        for linha in file:
            linha = linha.strip()
            if not linha or linha.startswith("—"):
                continue

            if linha.startswith("#"):
                secao = linha.split()[0][1:]
                continue

            partes = linha.split()

            match secao:
                case "cc":
                    turma, cursos = partes[0], partes[1:]
                    dados["cc"][turma] = cursos
                
                case "olw":
                    if partes:
                        dados["olw"].append(partes[0])
                
                case "dsd":
                    docente, cursos = partes[0], partes[1:]
                    dados["dsd"][docente] = cursos

                case "tr":
                    docente, restricoes = partes[0], list(map(int, partes[1:]))
                    dados["tr"][docente] = restricoes

                case "rr":
                    curso, sala = partes[0], partes[1]
                    dados["rr"][curso] = sala

                case "oc":
                    curso, idx = partes[0], int(partes[1])
                    dados["oc"][curso] = idx

                case _:
                    pass

    return dados

# Read data and initialize

In [8]:

dados = lerDados("ClassTT_01_tiny.txt")
print("A carregar dados")
print(dados)

# Inicializar problema CSP
problem = Problem()
blocos = list(range(1, 21))  # 20 blocos semanais

A carregar dados
{'cc': {'t01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'], 't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'], 't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']}, 'olw': [], 'dsd': {'jo': ['UC11', 'UC21', 'UC22', 'UC31'], 'mike': ['UC12', 'UC23', 'UC32'], 'rob': ['UC13', 'UC14', 'UC24', 'UC33'], 'sue': ['UC15', 'UC25', 'UC34', 'UC35']}, 'tr': {'mike': [13, 14, 15, 16, 17, 18, 19, 20], 'rob': [1, 2, 3, 4], 'sue': [9, 10, 11, 12, 17, 18, 19, 20]}, 'rr': {'UC14': 'Lab01', 'UC22': 'Lab01'}, 'oc': {'UC21': 2, 'UC31': 2}}


# Create variables (each UC has 1(olw) or 2 classes per week)

In [9]:
for turma, cursos in dados["cc"].items():
    for curso in cursos:
        if curso not in dados.get("olw", []):
            problem.addVariable(f"{curso}_1", blocos)
            problem.addVariable(f"{curso}_2", blocos)
            problem.addConstraint(lambda x, y: x != y, (f"{curso}_1", f"{curso}_2")) # Garantir que as duas aulas são em blocos diferentes
        else:
            problem.addVariable(f"{curso}_1", blocos) #UC Owls têm apenas 1 aula por semana

# Teacher restrictions (a teacher cannot teach two classes at the same time)

In [10]:
for docente, cursos in dados["dsd"].items():
    variaveis = []
    for curso in cursos:    #array com todas UCs de um docente UC_1 e UC_2 ou apenas UC_1 se for owl
        if curso not in dados.get("olw", []):
            variaveis.extend([f"{curso}_1", f"{curso}_2"])
        else:
            variaveis.append(f"{curso}_1")
    problem.addConstraint(AllDifferentConstraint(), variaveis) # Constraint para que não tenha UCs repetidas


# Teacher unavailability restrictions

In [11]:
for docente, indisponiveis in dados.get("tr", {}).items():
    cursos = dados["dsd"].get(docente, []) #UCs do docente
    for curso in cursos:
        if curso not in dados.get("olw", []):
            for i in [1, 2]:
                problem.addConstraint(
                    lambda bloco, indisponiveis=indisponiveis: bloco not in indisponiveis, #Bloco = variável , indisponiveis = lista de blocos indisponiveis (Dominio da variável)
                    (f"{curso}_{i}",)
                )
        else:
            problem.addConstraint(
                lambda bloco, indisponiveis=indisponiveis: bloco not in indisponiveis, #Compara UC_x com blocos indisponiveis para o docente que leciona a UC
                (f"{curso}_1",)
            )

# Class restrictions (each class can only have one lesson per block)

In [ ]:
for turma, cursos in dados["cc"].items():
    variaveis = []
    for curso in cursos:    #array com todas UCs de uma turma UC_1 e UC_2 ou apenas UC_1 se for owl
        if curso not in dados.get("olw", []):
            variaveis.extend([f"{curso}_1", f"{curso}_2"])
        else:
            variaveis.append(f"{curso}_1")
    problem.addConstraint(AllDifferentConstraint(), variaveis) # Constraint para que não tenha UCs repetidas

# Room restrictions (only for UCs with assigned room)

In [13]:
salas = {}

for curso, sala in dados.get("rr", {}).items():
    if sala not in salas:
        salas[sala] = []
    if curso not in dados.get("olw", []):
        salas[sala].extend([f"{curso}_1", f"{curso}_2"])
    else:
        salas[sala].append(f"{curso}_1")


# Add constraint : Each room can only have one class per block

In [14]:

# Cada sala só pode ter uma aula por bloco
for sala, variaveis in salas.items():
    problem.addConstraint(AllDifferentConstraint(), variaveis) # Constraint para que não tenha UCs com a mesma sala não tenham o mesmo bloco

# Solution

In [ ]:

# ==========================================
# Obter primeira solução
# ==========================================
solucao = problem.getSolution()

# ==========================================
# Transformar em JSON final legível
# ==========================================
if solucao:
    print("\nSolução encontrada:")
    print(json.dumps(solucao, indent=4))
else:
    print("Nenhuma solução encontrada")
